# Import Lib

In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from keras.models import Model
from keras.layers import *
from six.moves import range

Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '-',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

# Data Generation

In [13]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # if a<b, swap the value
    if a<b:
        temp=a
        a=b
        b=temp
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [14]:
print(questions[:5], expected[:5])

['88-33  ', '77-9   ', '20-5   ', '604-9  ', '25-6   '] ['55  ', '68  ', '15  ', '595 ', '19  ']


# Processing

In [15]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [16]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Data:
(60000, 7, 12)
(60000, 4, 12)


In [17]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False  True False False False False False False False
   False]
  [False False  True False False False False False False False False
   False]
  [False False False False False False False False False False  True
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False False  True False
   False]
  [False False False False False False False  True False False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False False False False
    True]
  [False False False  True False False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False  True False False False
   False]
  [False False  True False False False False False False False False
   False]
  [ True False False False False False Fal

# Build Model

In [18]:
x_train.shape[1:]

(7, 12)

In [20]:
print('Build model...')

##### Build your own model here ############
def my_model(input_data):
    
    model = Sequential()
    model.add(layers.LSTM(HIDDEN_SIZE, input_shape=(input_data[1], input_data[2]), name="lstm_1") )
    model.add(layers.RepeatVector(4, name="repeat_vector"))
    model.add(layers.LSTM(HIDDEN_SIZE, return_sequences=True, name="lstm_2"))
    model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax'), name="time_distributed"))

    model.summary()
    
    return model

Build model...


In [21]:
x_train.shape[1]

7

In [22]:
model = my_model(x_train.shape)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed (TimeDistri (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [23]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 4s 228us/step - loss: 1.8837 - acc: 0.3282 - val_loss: 1.7431 - val_acc: 0.3489
Q 867-15  T 852  ☒ 15  
Q 491-27  T 464  ☒ 14  
Q 771-89  T 682  ☒ 15  
Q 376-18  T 358  ☒ 15  
Q 281-270 T 11   ☒ 15  
Q 502-3   T 499  ☒ 15  
Q 227-71  T 156  ☒ 15  
Q 357-40  T 317  ☒ 15  
Q 841-79  T 762  ☒ 14  
Q 352-71  T 281  ☒ 15  

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 158us/step - loss: 1.7160 - acc: 0.3584 - val_loss: 1.6913 - val_acc: 0.3638
Q 868-3   T 865  ☒ 28  
Q 112-77  T 35   ☒ 11  
Q 849-275 T 574  ☒ 24  
Q 613-182 T 431  ☒ 11  
Q 879-781 T 98   ☒ 22  
Q 188-45  T 143  ☒ 14  
Q 425-81  T 344  ☒ 140 
Q 399-96  T 303  ☒ 28  
Q 99-1    T 98   ☒ 28  
Q 539-3   T 536  ☒ 24  

-------------------

18000/18000 [==============================] - 3s 160us/step - loss: 1.1801 - acc: 0.5659 - val_loss: 1.1633 - val_acc: 0.5683
Q 926-27  T 899  ☒ 996 
Q 336-47  T 289  ☒ 257 
Q 891-8   T 883  ☒ 881 
Q 79-65   T 14   ☒ 10  
Q 285-5   T 280  ☒ 257 
Q 904-405 T 499  ☒ 456 
Q 472-194 T 278  ☒ 351 
Q 849-52  T 797  ☒ 786 
Q 316-94  T 222  ☒ 257 
Q 885-350 T 535  ☒ 555 

--------------------------------------------------
Iteration 15
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 163us/step - loss: 1.1425 - acc: 0.5811 - val_loss: 1.1291 - val_acc: 0.5856
Q 42-18   T 24   ☒ 22  
Q 375-284 T 91   ☒ 14  
Q 360-8   T 352  ☒ 356 
Q 32-4    T 28   ☒ 23  
Q 596-404 T 192  ☒ 189 
Q 370-12  T 358  ☒ 344 
Q 673-670 T 3    ☒ 1   
Q 718-442 T 276  ☒ 261 
Q 99-1    T 98   ☒ 99  
Q 422-9   T 413  ☒ 412 

--------------------------------------------------
Iteration 16
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 165us/step - loss: 0.3192 - acc: 0.9025 - val_loss: 0.3613 - val_acc: 0.8740
Q 526-16  T 510  ☑ 510 
Q 28-15   T 13   ☑ 13  
Q 913-84  T 829  ☑ 829 
Q 383-89  T 294  ☑ 294 
Q 698-479 T 219  ☒ 229 
Q 387-338 T 49   ☒ 6   
Q 905-34  T 871  ☒ 872 
Q 222-6   T 216  ☑ 216 
Q 279-272 T 7    ☒ 4   
Q 790-69  T 721  ☑ 721 

--------------------------------------------------
Iteration 43
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 165us/step - loss: 0.2957 - acc: 0.9125 - val_loss: 0.3127 - val_acc: 0.8939
Q 964-215 T 749  ☑ 749 
Q 964-415 T 549  ☑ 549 
Q 48-11   T 37   ☑ 37  
Q 842-22  T 820  ☑ 820 
Q 783-35  T 748  ☑ 748 
Q 944-829 T 115  ☑ 115 
Q 116-71  T 45   ☒ 55  
Q 907-641 T 266  ☑ 266 
Q 108-27  T 81   ☒ 82  
Q 844-44  T 800  ☒ 700 

--------------------------------------------------
Iteration 44
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 162us/step - loss: 0.0815 - acc: 0.9773 - val_loss: 0.1167 - val_acc: 0.9573
Q 182-76  T 106  ☑ 106 
Q 883-53  T 830  ☑ 830 
Q 875-21  T 854  ☑ 854 
Q 324-317 T 7    ☒ 2   
Q 358-75  T 283  ☑ 283 
Q 775-5   T 770  ☑ 770 
Q 60-22   T 38   ☑ 38  
Q 384-287 T 97   ☒ 18  
Q 715-19  T 696  ☒ 796 
Q 448-2   T 446  ☑ 446 

--------------------------------------------------
Iteration 71
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 162us/step - loss: 0.0666 - acc: 0.9840 - val_loss: 0.1152 - val_acc: 0.9595
Q 379-140 T 239  ☑ 239 
Q 395-15  T 380  ☑ 380 
Q 194-67  T 127  ☑ 127 
Q 978-9   T 969  ☑ 969 
Q 519-3   T 516  ☑ 516 
Q 426-340 T 86   ☑ 86  
Q 287-50  T 237  ☑ 237 
Q 718-442 T 276  ☑ 276 
Q 823-10  T 813  ☑ 813 
Q 22-19   T 3    ☑ 3   

--------------------------------------------------
Iteration 72
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=====

18000/18000 [==============================] - 3s 169us/step - loss: 0.0241 - acc: 0.9964 - val_loss: 0.0749 - val_acc: 0.9736
Q 318-96  T 222  ☑ 222 
Q 110-31  T 79   ☒ 78  
Q 190-18  T 172  ☑ 172 
Q 512-24  T 488  ☑ 488 
Q 255-124 T 131  ☒ 121 
Q 948-322 T 626  ☑ 626 
Q 809-556 T 253  ☑ 253 
Q 429-192 T 237  ☑ 237 
Q 986-92  T 894  ☑ 894 
Q 279-27  T 252  ☑ 252 

--------------------------------------------------
Iteration 99
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 169us/step - loss: 0.0231 - acc: 0.9967 - val_loss: 0.0714 - val_acc: 0.9740
Q 68-52   T 16   ☑ 16  
Q 891-8   T 883  ☑ 883 
Q 778-374 T 404  ☑ 404 
Q 92-67   T 25   ☑ 25  
Q 956-188 T 768  ☑ 768 
Q 607-23  T 584  ☑ 584 
Q 530-15  T 515  ☑ 515 
Q 129-58  T 71   ☑ 71  
Q 817-34  T 783  ☑ 783 
Q 525-320 T 205  ☑ 205 


# Testing

In [27]:
def vectorization(ary_x, ary_y):
    print('Vectorization...')
    x = np.zeros((len(test_x), MAXLEN, len(chars)), dtype=np.bool)
    y = np.zeros((len(test_y), DIGITS + 1, len(chars)), dtype=np.bool)
    for i, sentence in enumerate(test_x):
        x[i] = ctable.encode(sentence, MAXLEN)
    for i, sentence in enumerate(test_y):
        y[i] = ctable.encode(sentence, DIGITS + 1)
    return x, y

In [28]:
def evaluation_test(x_test, y_test, model):
    x_val, y_val = vectorization(test_x, test_y)
    for i in range(len(y_val)):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

In [29]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
# test_x = ["555-175", "860-7  ", "340-29 "]
# test_y = ["380 ", "853 ", "311 "] 

# evaluation_test(test_x, test_y, model)

#####################################################
testing_callback = model.evaluate(x=test_x, y=test_y, batch_size=1, verbose=1)
for i in range(len(model.metrics_names)):
        print(model.metrics_names[i]+" : "+str(testing_callback[i]))

MSG : Prediction


AttributeError: 'str' object has no attribute 'ndim'